In [19]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

In [20]:
response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
docs = response['response']['docs']
url_list = []
for item in docs:
    url_list.append(item['web_url'])
article_list = []
url_list


['https://www.nytimes.com/2020/12/16/technology/facebook-takes-the-gloves-off-in-feud-with-apple.html',
 'https://www.nytimes.com/2020/12/13/business/media/apple-gawker-tim-cook.html',
 'https://www.nytimes.com/2020/12/23/business/dealbook/trump-stimulus-veto.html',
 'https://www.nytimes.com/2020/12/01/technology/amazon-apple-chips-intel-arm.html',
 'https://www.nytimes.com/2020/12/17/business/dealbook/tech-apple-facebook-fight.html',
 'https://www.nytimes.com/2020/12/17/technology/google-antitrust-monopoly.html',
 'https://www.nytimes.com/2020/12/15/technology/big-tech-regulation-europe.html',
 'https://www.nytimes.com/2020/12/14/technology/big-tech-lobbying-europe.html',
 'https://www.nytimes.com/2020/12/09/technology/personaltech/amazon-halo-review.html',
 'https://www.nytimes.com/2020/11/18/technology/apple-app-store-fee.html']

In [21]:
text = []
title = []
abstract = []
for url in url_list:
    time.sleep(0.5)
    headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
    article = requests.get(url, headers = headers)
    soup = bs4.BeautifulSoup(article.content, 'html.parser')
    article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
    abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
    title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
    temp = []
    title.append(title_text_h1.text)
    abstract.append(abstract_text_p.text)
    
    for item in article_text_p:
        temp.append(item.text)
    space = ' '
    article_text = space.join(temp)
    text.append(article_text)

In [22]:
len(text)

10

In [23]:
# Word Tokenization to sentences

tokenized_by_sentence = []
for num in range(len(text)):
    del_quo = re.sub(",”", " ", text[num])
    del_quo_2  = re.sub("”", " ", del_quo)
    del_quo_3 = re.sub("“", "", del_quo_2)
    text_token = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!|;|”)\s", del_quo_3)
    text_token.insert(0, abstract[num])
    text_token.insert(0, title[num])
    tokenized_by_sentence.append(text_token)

tokenized_by_sentence[0]

['Facebook Takes the Gloves Off in Feud With Apple',
 'The social network said it opposed changes that Apple was making to the tracking of apps and would provide information for an antitrust complaint against the iPhone maker.',
 'For years, signs of discord have brewed between Facebook and Apple.',
 'Their chief executives, Apple’s Tim Cook and Facebook’s Mark Zuckerberg, have periodically taken thinly veiled shots at each other.',
 'If they’re making money mainly by collecting gobs of personal data, I think you have a right to be worried  Mr. Cook said of companies like Facebook in 2014.',
 'In turn, Mr. Zuckerberg has retorted: You think because you’re paying Apple that you’re somehow in alignment with them?',
 'If you were in alignment with them, then they’d make their products a lot cheaper.',
 ' But now Apple is making changes that threaten Facebook’s business — and the fight has intensified.',
 'Early next year, Apple plans to start requiring iPhone owners to explicitly choose w

In [24]:
# Goal of this part: Read through all positive / negative tweets, normalize and remove unnecessary words from tweets, then create actual dictionary-like to use for our dataset
import nltk
nltk.download('punkt')
nltk.download('twitter_samples')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [25]:
from nltk.corpus import twitter_samples

pos_tweet = twitter_samples.tokenized('positive_tweets.json')
neg_tweet = twitter_samples.tokenized('negative_tweets.json')

In [26]:
# Convert all complex part-of-speech to basic words
# List of part-of-speech is in this link: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
# WordNetLemmatizer has a function lemmatize where you can convert complex part of speech words into basic forms
# Things to consider:
#   Remove all unnecessary words from normalized_neg_tweets / normalized_pos_tweets
#   1. Remove mentions(starts with @)
#   2. Remove links (starts with https:// or http:// )
#   3. Remove punctuation (starts with ! or ?)
#   4. Remove Stop-Words (words that do have little to no meaning and does not affect the context of the sentence) to make our dataset more concise
# Note that we are keeping emoji (i.e. :) or :( . That is because these emojis do actually show sentiment of the text context)
# If words are DETERMINERS (DT), COORDINATING CONJUCTIONS (CC), PREPOSITIONS (IN), PERSONAL / POSSESSIVE PRONOUNS (PRP / PRP$), or WH-PRONOUNS (WP) WH-ADVERB(WRB), we remove it (consider as Stop words)
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import string

normalizer = WordNetLemmatizer()
punctuation = ['!', '"', '#', '$' ,'%', '&', "'",'(',')','*','+',',','-','.',':',';','<','=','>','?','@','[','],''^','_','`','{','|','}','~']

def determiners(word):
    for i in word:
        if i in punctuation:
            return False
    if word.startswith('@') or word.startswith('!') or word.startswith('?') or word.startswith('https://') or word.startswith('http://'):
        return False
    else:
        return True

def normalize(tweet_list):
    normalized_tweet = []
    for tweet in tweet_list:
        sentence = []
        for token, tag in pos_tag(tweet):
            # For Complex Noun words:
            if tag.startswith('NN'):
                new_tag = 'n'
            # For Complex Verb words
            elif tag.startswith('VB'):
                new_tag = 'v'
            # For stop-words
            elif tag.startswith('DT') or tag.startswith('CC') or tag.startswith('IN') or tag.startswith('PRP') or tag.startswith('PRP$') or tag.startswith('WP') or tag.startswith('WRB'):
                continue 
            # Every other words, convert them into adjective (pos = 'a')
            else:
                new_tag = 'a'
            sentence.append(normalizer.lemmatize(token, new_tag))
        new_sentence = [ word for word in sentence if determiners(word) ]
        normalized_tweet.append(new_sentence)
    return normalized_tweet

normalized_pos_tweets = normalize(pos_tweet)
normalized_neg_tweets = normalize(neg_tweet)


In [27]:
# filter out the stop words
# borrowed list of stop words from https://github.com/kavgan/stop-words/blob/master/terrier-stop.txt 

filepath = open("terrier-stop.txt", "r")
stop_words = filepath.read().split("\n")

In [28]:
# Now, store all positive / negative words into dictionary so it can be used as a guide for calculating sentiment for sentences
pos_words_dict = {}
neg_words_dict = {}

# Store all words into dictionary
for tweet in normalized_pos_tweets:
    for word in tweet:
        if word in stop_words:
            continue
        elif word in pos_words_dict:
            temp = pos_words_dict[word.lower()]
            temp += 1
            pos_words_dict[word.lower()] = temp
        else:
            pos_words_dict[word.lower()] = 1

for tweet in normalized_neg_tweets:
    for word in tweet:
        if word in stop_words:
            continue
        elif word in neg_words_dict:
            temp = neg_words_dict[word.lower()]
            temp += 1
            neg_words_dict[word.lower()] = temp
        else:
            neg_words_dict[word.lower()] = 1


In [29]:
# remove all emojis and leave only roman alphabets
pos_df = pd.DataFrame({'word': list(pos_words_dict.keys()), 'frequency': list(pos_words_dict.values())})
cleaned_pos_df = pos_df.loc[pos_df['word'].str.isalpha()]

neg_df = pd.DataFrame({'word': list(neg_words_dict.keys()), 'frequency': list(neg_words_dict.values())})
cleaned_neg_df = neg_df.loc[neg_df['word'].str.isalpha()]

In [30]:
# assign scales according to frequency between absolute values 1 to 5 (-1 to -5 for negative words)
cleaned_pos_df['scale'] = cleaned_pos_df['frequency'] * (4 / cleaned_pos_df['frequency'].max()) + 1
cleaned_neg_df['scale'] = cleaned_neg_df['frequency'] * (-4 / cleaned_neg_df['frequency'].max()) - 1

In [59]:
# tokenize and lametize the article from new york times

def tokenizer_myself(given_articles):
    tokenized_result = []
    for article_iter in given_articles: 
        temp = []
        for sentence in article_iter:
            tokenized_sentence = sentence.split(" ")
            if tokenized_sentence:
                temp.append(tokenized_sentence)
        new_temp = normalize(temp)
        tokenized_result.append(new_temp)
    return tokenized_result

tokenized_by_sentence_new = tokenizer_myself(tokenized_by_sentence)

tokenized_by_sentence_new

IndexError: string index out of range

['Facebook', 'Takes', 'the', 'Gloves', 'Off', 'in', 'Feud', 'With', 'Apple']